In [2]:
import numpy as np

import monotonic_function
import tensorflow as tf
from tensorflow import keras
from keras import layers
import sklearn
import sklearn.metrics
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()


2022-11-27 12:00:29.057665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-27 12:00:29.287177: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-27 12:00:29.328194: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-27 12:00:29.328215: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [124]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_test = x_train[..., np.newaxis]
x_train = x_train[..., np.newaxis]
y_test = y_test[..., np.newaxis]
y_train = y_train[..., np.newaxis]

def group_in_two(y):
    y[y < 5] = 0
    y[y >= 5] = 1

group_in_two(y_train)
group_in_two(y_test)


In [125]:
input_shape = x_train.shape[1:3]
model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(2, activation=None),
    layers.Softmax()]
)
def loss_func(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    y_true = y_true[:, 0]
    eps = 1e-6
    p0 = tf.clip_by_value(y_pred[:, 0] + eps, 0, 1)
    p1 = tf.clip_by_value(y_pred[:, 1] + eps, 0, 1)
    loss = -tf.math.xlogy(y_true, p1) - tf.math.xlogy(1 - y_true, p0)
    loss /= tf.math.log(2)
    return loss

optimizer = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(loss=loss_func, optimizer=optimizer)


In [131]:
model.optimizer.learning_rate = 0.0001
model.fit(x_train, y_train, batch_size=60000, epochs=400)

Epoch 1/400
1/1 [==============================] - 0s 70ms/step - loss: 1.5761
Epoch 2/400
1/1 [==============================] - 0s 36ms/step - loss: 1.5394
Epoch 3/400
1/1 [==============================] - 0s 40ms/step - loss: 1.5197
Epoch 4/400
1/1 [==============================] - 0s 37ms/step - loss: 1.5049
Epoch 5/400
1/1 [==============================] - 0s 36ms/step - loss: 1.4918
Epoch 6/400
1/1 [==============================] - 0s 36ms/step - loss: 1.4787
Epoch 7/400
1/1 [==============================] - 0s 37ms/step - loss: 1.4652
Epoch 8/400
1/1 [==============================] - 0s 36ms/step - loss: 1.4552
Epoch 9/400
1/1 [==============================] - 0s 36ms/step - loss: 1.4487
Epoch 10/400
1/1 [==============================] - 0s 33ms/step - loss: 1.4455
Epoch 11/400
1/1 [==============================] - 0s 37ms/step - loss: 1.4427
Epoch 12/400
1/1 [==============================] - 0s 39ms/step - loss: 1.4394
Epoch 13/400
1/1 [==============================]

In [132]:
y_pred = model(x_train).numpy()
scores = y_pred[:, 1]
y_pred_bin = np.argmax(y_pred, axis=1)
loss = loss_func(y_train, y_pred).numpy()
np.average(loss)

1.3807496

In [133]:
sklearn.metrics.accuracy_score(y_train, y_pred_bin)

0.8878

In [134]:
model.evaluate(x_train, y_train)

1875/1875 [==============================] - 1s 459us/step - loss: 1.3808


1.3807531595230103

In [13]:
y_true = np.array([False, False, True, False, True])
scores = np.array([0.5, 0.2, 0.2, 0.2, 0.3])
l = [y for x, y in sorted(zip(-scores, y_true))]
# was sorted by -scores to ensure descending order of y_true for equal scores
l = l[::-1]
s = list(zip(*sorted(zip(-scores, y_true))))[1][::-1]


In [26]:
sc = np.array([0.8, 0.6, 0.3, 0.4])
inds = np.argsort(sc)
s = np.zeros_like(inds)
sc[inds] = np.array([4, 5, 6, 7])